In [49]:
import pandas as pd
import datasets
import sys
import os
import json
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
import utils
aspects = [ 'actionability', 'grounding_specificity', 'verifiability', 'helpfulness']

# Load the dataset
data_1 = pd.read_excel('../chatgpt/outputs/main_data_batch_1_results.xlsx', sheet_name=None)
data_2 = pd.read_excel('../chatgpt/outputs/main_data_batch_2_results.xlsx', sheet_name=None)
data_3 = pd.read_excel('../chatgpt/outputs/main_data_batch_3_results.xlsx', sheet_name=None)
data_4 = pd.read_excel('../chatgpt/outputs/main_data_batch_4_results.xlsx', sheet_name=None)

data =  {}

### combine the two batches
for key in data_1.keys():
    if key in data_2.keys() and key in data_3.keys() and key in data_4.keys():
        data[key] = pd.concat([data_1[key], data_2[key], data_3[key], data_4[key]])
    elif key in data_2.keys() and key in data_3.keys():
        data[key] = pd.concat([data_1[key], data_2[key], data_3[key]])
        

## remove the verifiability_extraction sheet
data.pop('verifiability_extraction')

## Remove entries that has NA in the score or rationale columns
for key in data.keys():
    score = f'chatgpt_{key}_rationale'
    ratioanele = f'chatgpt_{key}_rationale'
    data[key] = data[key].dropna(subset=[score, ratioanele])
    

In [47]:
# ## take rpoints that are in all keys but not in helpfullness
# rpoints = set(data['actionability']['review_point'])
# for key in data.keys():
#     if key != 'helpfulness':
#         rpoints = rpoints.intersection(set(data[key]['review_point']))
# print(f"Number of review points: {len(rpoints)}")

# ## remove helpfulness points from rpoints
# rpoints = rpoints - set(data['helpfulness']['review_point'])

# dd = data['actionability'][data['actionability']['review_point'].isin(rpoints)][:50]

# print(f"Number of review points: {len(dd)}")

# dd.to_excel('/home/abdelrahman.sadallah/mbzuai/review_rewrite/chatgpt/main_data/batches/batch_4.xlsx', sheet_name='helpfulness', index=False)


In [50]:
for aspect in aspects:
    data[aspect] = data[aspect].drop_duplicates(subset=['review_point'], keep='first')

# Step 1: Find common review points across all dataframes
rpoints = set(data['helpfulness']['review_point'])  # Start with one aspect
for key in data.keys():
    rpoints &= set(data[key]['review_point'])  # Keep only review points that exist in all dataframes

# Step 2: Filter dataframes to keep only common review points
for key in data.keys():
    data[key] = data[key][data[key]['review_point'].isin(rpoints)]  # Filter to common review points

# Step 3: Sort and reset index to ensure order is consistent
for key in data.keys():
    data[key] = data[key].sort_values(by='review_point').reset_index(drop=True)  # Sort and reset index

# Step 4: Use 'helpfulness' as a reference to pick the final 10,000 review points
selected_review_points = data['helpfulness']['review_point'][:10000].tolist()  # Fixed reference set

# Step 5: Filter all dataframes to include only these exact review points and enforce the same order
for key in data.keys():
    data[key] = data[key][data[key]['review_point'].isin(selected_review_points)]  # Filter for consistency
    data[key] = data[key].sort_values(by='review_point').reset_index(drop=True)  # Final sorting

# Step 6: Validate alignment with assertions
for i, row in data['actionability'].iterrows():
    for aspect in data.keys():
        assert row['review_point'] == data[aspect].iloc[i]['review_point'], f"Mismatch at index {i} in aspect {aspect}"


In [51]:
### make sure the data is in the same order
for i,row in data['actionability'].iterrows():
    for aspect in aspects:
        
        assert row['review_point'] == data[aspect].iloc[i]['review_point']

In [52]:
for key in data.keys():
    print(key)
    print(len(data[key]))

helpfulness
10000
actionability
10000
grounding_specificity
10000
verifiability
10000


In [67]:
### convert the df to dataset and store for each aspect
for aspect in data.keys():

    data[aspect] = data[aspect].drop(columns=['prompt'])


    dataset = datasets.Dataset.from_pandas(data[aspect]).train_test_split(test_size=0.1)
    ## add new column for the prompt do in sepratately for th train and test
    
    # dataset['train'] = dataset['train'].map(lambda x: {'prompt': utils.get_prompt(x, aspect, task='train',generation_type='score_only')})
    # dataset['test'] = dataset['test'].map(lambda x: {'prompt': utils.get_prompt(x, aspect, task='test',generation_type='score_only')})
    dataset.push_to_hub(f'boda/review_evaluation_automatic_labels',config_name=aspect )
    




Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/4.08k [00:00<?, ?B/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

In [69]:

merged_df = []

### aggregate the different aspects in one dataset
for i,row in data['actionability'].iterrows():
    ## remove the prompt column
    # row.pop('prompt')
    failed = False
    for aspect in data.keys():
        if aspect == 'actionability':
            continue
        ### get the same row from the other aspects
        ## make sure the row exist
        if row['review_point'] not in data[aspect]['review_point'].values:
            print(f'row {row["id"]} not found in {aspect}')
            ## ignore this row
            failed = True
            continue

        aspect_row = data[aspect][data[aspect]['review_point'] == row['review_point']].iloc[0]
        row[f'chatgpt_{aspect}_score'] = aspect_row[f'chatgpt_{aspect}_score']
        row[f'chatgpt_{aspect}_rationale'] = aspect_row[f'chatgpt_{aspect}_rationale']
        ## if the score or rationale is none then ignore this row
        if pd.isna(aspect_row[f'chatgpt_{aspect}_score']):
            print(f'row {row["id"]} has no score in {aspect}')
            failed = True
            continue
        # row[f'{aspect}_prompt'] = aspect_row['prompt']
    if not failed:
        merged_df.append(row)
    
merged_df = pd.DataFrame(merged_df)


## convert the df to dataset and push to hub
dataset = datasets.Dataset.from_pandas(merged_df).train_test_split(test_size=0.1)
## add new column for the prompt
# dataset['train'] = dataset['train'].map(lambda x: {'prompt': utils.get_prompt(x, 'all', task='train',generation_type='score_only')})
# dataset['test'] = dataset['test'].map(lambda x: {'prompt': utils.get_prompt(x, 'all', task='test',generation_type='score_only')})
dataset.push_to_hub(f'boda/review_evaluation_automatic_labels',config_name='all' )

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/boda/review_evaluation_automatic_labels/commit/45293ff015720cfce9138c2daae83155dc15c59c', commit_message='Upload dataset', commit_description='', oid='45293ff015720cfce9138c2daae83155dc15c59c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/boda/review_evaluation_automatic_labels', endpoint='https://huggingface.co', repo_type='dataset', repo_id='boda/review_evaluation_automatic_labels'), pr_revision=None, pr_num=None)

In [63]:
for aspect in aspects:
    print(merged_df[f'chatgpt_{aspect}_score'].unique())

### get the rows that has the nan values
nan_rows = merged_df[merged_df.isna().any(axis=1)]


[2 5 3 4 1]
[5 4 3 1 2]
['X' '1' '3' '2' '4' '5']
[3 2 4 1 5]


In [64]:
nan_rows

,paper_id,venue,focused_review,review_point,id,chatgpt_actionability_score,chatgpt_actionability_rationale,chatgpt_helpfulness_score,chatgpt_helpfulness_rationale,chatgpt_grounding_specificity_score,chatgpt_grounding_specificity_rationale,chatgpt_verifiability_score,chatgpt_verifiability_rationale
820,ACL_2017_371_review,ACL_2017,NaN,"- Also, regarding section 3.3, please cite app...",209,5,The review point explicitly instructs the auth...,4,The review comment provides specific and actio...,5,The comment is fully grounded as it explicitly...,X,The comment requests the citation of appropria...
1379,ACL_2017_193_review,ACL_2017,NaN,- How l(u) is chosen in case of several childr...,339,4,"The review point raises a question about how ""...",3,The review comment identifies a specific area ...,2,The comment is not fully grounded as it does n...,X,The review point consists of a request for cla...
1711,ACL_2017_193_review,ACL_2017,NaN,"- In the converted graphs in figure 4, some ed...",343,5,The review point identifies a specific issue w...,5,The review comment provides specific and actio...,5,The comment is fully grounded as it explicitly...,3,The review point claims that some edges in the...
1939,ACL_2017_371_review,ACL_2017,NaN,- It's because if the largest phrase length is...,126,1,The review point describes a potential simplif...,2,The review comment attempts to provide an insi...,1,The comment discusses a potential simplificati...,X,The comment provides an explanation of how a m...
1946,ACL_2017_371_review,ACL_2017,NaN,- It's not true that the pRNN outperforms both...,199,5,The review point provides explicit instruction...,5,The review comment provides clear and actionab...,5,The comment is fully grounded as it explicitly...,4,The review point challenges the claim that the...
2955,ACL_2017_371_review,ACL_2017,NaN,- The evaluation on PTB (table 2) isn't a fair...,372,3,The review point highlights a potential issue ...,3,The review comment identifies a critical issue...,5,The comment is fully grounded as it explicitly...,4,The review point claims that the evaluation on...
3932,ACL_2017_193_review,ACL_2017,NaN,- While it is very complete to provide the pre...,131,4,"The review point suggests two actions: first, ...",4,The review comment provides actionable feedbac...,3,The comment suggests including an informal des...,3,The review point suggests that the paper would...
4362,ICLR_2022_2956,ICLR_2022,NaN,1) Is there any fundamental problem with dropo...,3027,2,The review point poses several questions regar...,2,The comment raises questions about the use of ...,3,The comment raises questions about the use of ...,X,The review point consists of questions asking ...
4396,NIPS_2018_45,NIPS_2018,NaN,1) Section 4.3 (roughly L267-270) briefly expl...,6188,3,The review point raises questions about the co...,3,The review comment highlights a specific secti...,5,The comment is fully grounded as it explicitly...,X,The review point consists of inquiries about t...
4424,ICLR_2022_2956,ICLR_2022,NaN,1) The hyperparameter tuning procedure seems p...,3140,3,The review point identifies a potential issue ...,4,The review comment identifies a critical issue...,5,The comment is fully grounded as it provides s...,4,The review comment raises a concern about the ...


In [65]:
# columns_to_keep = ['review_point','paper_id','venue','focused_review']
# all_datasets = {}
# for aspect in aspects:
#     aspect_dataset = []

#     aspect_data = all_data[all_data[f'{aspect}_label_type'] == 'gold']
#     aspcets_columns_to_keep = columns_to_keep + [f'{aspect}_label']
#     aspect_data = aspect_data[aspcets_columns_to_keep]

#     for i,row in aspect_data.iterrows():
#         row['prompt'] = utils.get_prompt(row, aspect, task='train',generation_type='score_only')
#         aspect_dataset.append(row)
#     aspect_dataset = pd.DataFrame(aspect_dataset)
#     aspect_dataset = datasets.Dataset.from_pandas(aspect_dataset)
#     if '__index_level_0__' in aspect_dataset.column_names:
#         aspect_dataset = aspect_dataset.remove_columns('__index_level_0__')

#     ## split the dataset to train and validation
#     aspect_dataset = aspect_dataset.train_test_split(test_size=0.1)
#     all_datasets[aspect] = aspect_dataset

    


KeyError: "Column prompt not in the dataset. Current columns in the dataset: ['paper_id', 'venue', 'focused_review', 'review_point', 'id', 'chatgpt_actionability_score', 'chatgpt_actionability_rationale', 'chatgpt_helpfulness_score', 'chatgpt_helpfulness_rationale', 'chatgpt_grounding_specificity_score', 'chatgpt_grounding_specificity_rationale', 'chatgpt_verifiability_score', 'chatgpt_verifiability_rationale', '__index_level_0__']"